In [1]:
import numpy as np
import sympy as sym
import copy

In [3]:

# Global variables (problem-specific data)
m = 2  # Number of basic variables
n = 2 * 5  # Total number of variables (k = 5)
eps = 1e-6  # Perturbation constant
Perturb = False  # Boolean flag for perturbation

# Initialize A, b, c (problem matrices)
A = np.array([[1, 2, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]])
b = np.array([1, 1])
c = np.array([-1, -2, 0, 0, 0, 0, 0, 0, 0, 0])

# Initial basic and nonbasic partitions
beta = [0, 1]  # Basic variables
eta = [2, 3, 4, 5, 6, 7, 8, 9]  # Nonbasic variables

# Extract submatrices for current partition
A_beta = A[:, beta]
A_eta = A[:, eta]
c_beta = c[beta]
c_eta = c[eta]

# Perturb the right-hand side (b) to avoid degeneracy
def pivot_perturb():
    global b, Perturb
    Perturb = True
    for i in range(m):
        for j in range(m):
            b[i] += A_beta[i, j] * eps**(j + 1)
    print('pivot_perturb() done')

# Perform the algebraic operations
def pivot_algebra():
    global xbar_beta, xbar_eta, ybar, cbar_eta, objval, ratios
    xbar_beta = np.linalg.solve(A_beta, b)
    xbar_eta = np.zeros(n - m)
    objval = np.dot(c_beta, xbar_beta)
    xbar = np.zeros(n)
    for i in range(m):
        xbar[beta[i]] = xbar_beta[i]
    for j in range(n - m):
        xbar[eta[j]] = xbar_eta[j]
    ybar = np.linalg.solve(A_beta.T, c[beta])
    cbar_eta = c_eta - np.dot(A_eta.T, ybar)
    ratios = np.inf * np.ones(m)
    print('pivot_algebra() done')
    print(f"Objective value: {objval}")
    print(f"Reduced costs: {cbar_eta}")

# Calculate ratios for a given nonbasic index
def pivot_ratios(j):
    global ratios, zbar
    if j > n - m - 1:
        print("Error: j is out of range.")
    else:
        A_etaj = A[:, eta[j]].copy()
        Abar_etaj = np.linalg.solve(A_beta, A_etaj)
        for i in range(m):
            if Abar_etaj[i] > 0:
                ratios[i] = xbar_beta[i] / Abar_etaj[i]
            else:
                ratios[i] = np.inf
        print(f"Ratios: {ratios}")
        zbar = np.zeros(n)
        for i in range(m):
            zbar[beta[i]] = -Abar_etaj[i]
        zbar[eta[j]] = 1
        print(f"Direction: zbar = {zbar}")

# Swap nonbasic eta_j in and basic beta_i out
def pivot_swap(j, i):
    global A_beta, A_eta, c_beta, c_eta
    if i > m - 1 or j > n - m - 1:
        print("Error: j or i is out of range. Swap not accepted.")
    else:
        save = beta[i]
        beta[i] = eta[j]
        eta[j] = save
        A_beta = A[:, beta].copy()
        A_eta = A[:, eta].copy()
        c_beta = c[beta].copy()
        c_eta = c[eta].copy()
        print(f"Swap accepted --- new partition:")
        print(f"eta: {eta}")
        print(f"beta: {beta}")
        print("*** MUST APPLY pivot_algebra()! ***")

# Example of one complete pivot step
def pivot_step(j):
    pivot_ratios(j)
    i = np.argmin(ratios)  # Pick the smallest ratio
    if ratios[i] == np.inf:
        print("Unbounded solution.")
    else:
        pivot_swap(j, i)
        pivot_algebra()

# Initial algebraic setup
pivot_algebra()

# Uncomment the perturbation line to introduce perturbation
pivot_perturb()  # Uncomment this line for perturbation

# Example pivot step (start with j = 0 for demonstration)
pivot_step(0)  # First pivot step with eta_j (try with j = 0)


pivot_algebra() done
Objective value: -1.0
Reduced costs: [1. 0. 0. 0. 0. 0. 0. 0.]
pivot_perturb() done
Ratios: [-1. inf]
Direction: zbar = [-1. -0.  1.  0.  0.  0.  0.  0.  0.  0.]
Swap accepted --- new partition:
eta: [0, 3, 4, 5, 6, 7, 8, 9]
beta: [2, 1]
*** MUST APPLY pivot_algebra()! ***
pivot_algebra() done
Objective value: -2.0
Reduced costs: [-1.  2.  0.  0.  0.  0.  0.  0.]
